Краткое описание:
* Датасет не сбалансирован. Решил дополнить выборку. Удалось найти порно заголовки только на английском. Так исходные заголовки преимущественно на русском воспользовался yandex API Translate. Переводил партиями, ибо очень долго. В итоге удалось напереводить 100к порно заголовков. Большинство исходных данных мало напоминают заголовки, тем не менее наиболее близкое, что удалось найти. Датасет из проекта наташа corpus "новости rt". Url -ы удалось найти на  кегл соревновании на классификацию сайтов. Там присутствовали url порно сайтов, но их примерно 36-37к. Т.к. поэтому для 100к примеров urls заполнялись следующим образом: 36к с кегл соревнования; исходные urls (16.7к) + 25к выборка с повторением из исходных и остаток заполнялся урлами с кегла (без повторений). Попробывал предсказать по своей выборке исходную выборку. Все тривиальные преобразования: конкатенация датафреймов, переинования столбцов и тд опущены.

* Обработка данных: Заголовки:удалялись лишние, на мой взгляд, символы и числа. Заголовки лемматезировались (использовал наташу,на мой взгляд, работает лучше с русскими словами, чем pymorphy2, в особенности с порно словами. Форма слова выбиралась по наиболее лучшим тегам. Далее из полученных списков токенов удалялись стоп слова и слова короче 2 символов. Пустым строкам присваивалось '#nan'
 Urls: сплитились по мусорным, на мой взгляд, символам. Удалялись com ru www, а для url с кегла еще и http//:. Каждое слово списка билось на ngram-ы.

Функции перевода, обработки закоментированы, т.к. считаются очень долго

    https://sexualitics.github.io/ здесь брал порно заголовки
    https://www.kaggle.com/shawon10/url-classification-dataset-dmoz - здесь брал урлы

In [ ]:
!pip install natasha
!pip install catboost
!pip install --upgrade gensim

     |████████████████████████████████| 34.4 MB 29 kB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 41 kB 107 kB/s 
     |████████████████████████████████| 8.2 MB 14.7 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=66be8b38baa5386d20b59207c4983fb31aea70311aa0733d19f3af629c108160
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0
     |████████████████████████████████| 76.3 MB 20 kB/s 


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')

Процедура перевода порнозаголовков

In [ ]:
# df_porn = pd.read_csv('xnxx.csv')
# df_porn['translation'] = " "
# feat = {'title', 'translation'}
# df_porn = df_porn[feat]
# df_porn.head()

In [ ]:
# from yandex.Translater import Translater
# tr = Translater()
# tr.set_key('trnsl.1.1.20211104T223538Z.f24011fa7f0c9f9c.8ba29501f3e9e4b83109f0a3ebffc5ceb0fe859a')
# tr.set_from_lang('en')
# tr.set_to_lang('ru')

# for i in tqdm(range(70000, 100000)):
#     tr.set_text(df_porn.title[i])
#     df_porn.translation[i] = tr.translate()

In [ ]:
# from tqdm.notebook import tqdm
# for i in tqdm(range(70000, 100000)):
#     tr.set_text(df_porn.title[i])
#     df_porn.translation[i] = tr.translate()

Токенизация url на ngram

In [ ]:
# from nltk.tokenize import SyllableTokenizer
# tk = SyllableTokenizer()

# def urls_tokenize(df):
#     stop_ngram = ['ru', 'com', 'www', '', ' ']
#     for i in tqdm(range(df.shape[0])):
#         split = re.split('[^A-Za-z0-9]+', df.url[i])
#         df.url[i] = [token for token in split if token not in stop_ngram]
    
#     for i in tqdm(range(df.shape[0])):
#         ngrams =[]
#         for word in df.url[i]:
#             ngrams = ngrams + tk.tokenize(word)
#         df.url[i] = ngrams
#     return df
    

In [ ]:
# df = urls_tokenize(df)

Токенизация заголоков. Осталяю только буквы, перевожу в нижний регистр. Лемматизирую наташей. Удаляю стоп слова. Удаляю слова короче 2 символов

In [ ]:
# from nltk.corpus import stopwords
# from natasha import (
#     Segmenter,
#     NewsEmbedding,
#     NewsMorphTagger, 
#     MorphVocab,
#     Doc,
# )
# segmenter = Segmenter()
# emb = NewsEmbedding()
# morph_tagger = NewsMorphTagger(emb)
# morph_vocab = MorphVocab()


# def preprocessing_data(df):
    
#     def my_filter(text):
#         return re.findall("[а-яА-ЯёЁa-zA-Z]+", text.lower())
#     print("filter start: ")
#     for i in tqdm(range(df.shape[0])):
#         df.title[i] = my_filter(df.title[i]);
#     print("filter finish: ")
#     for i in tqdm(range(df.shape[0])):
#         string = ' '.join(df.title[i])
#         df.title[i] = string
    
#     def lemmatizer(text):
#         doc = Doc(text)
#         doc.segment(segmenter)
#         doc.tag_morph(morph_tagger)
#         for token in doc.tokens:
#             token.lemmatize(morph_vocab)
#         return [_.lemma for _ in doc.tokens]
    
#     print("lemmatizer start: ")
#     for i in tqdm(range(df.shape[0])):
#         df.title[i] = lemmatizer(df.title[i])
#     print("lemmatizer finish:")
    
#     print("stop words start")
#     for i in tqdm(range(df.shape[0])):
#         clear = [token for token in df.title[i] if token not in stopwords.words(['russian', 'english'])]
#         df.title[i] = clear
#     print("stop words finish")
#     print('delete words < 2 start: ')
#     for i in tqdm(range(df.shape[0])):
#         clear = [token for token in df.title[i] if len(token) > 2 ]
#         df.title[i] = clear
#     print('delete words < 2 finisha: ')
#     return df

In [ ]:
# df = preprocessing_data(df)
# df

# Тестирование собранной выборки

Оценка качества на исходной выборке. Тестирую только на очищенных данных, тк. в данных коррелированный шум, т.е. после препроцессинга и токенизации

In [ ]:
from sklearn.metrics import f1_score
from ast import literal_eval
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier


preprocessor = ColumnTransformer([
    ('url_tokens', TfidfVectorizer(analyzer='word'), 'url_tokens'),
    ('title_tokens', TfidfVectorizer(analyzer='word'), 'title_tokens'),
])


model = Pipeline([
    ('preprocessor', preprocessor),
    ('Linear_model', LogisticRegression(class_weight='balanced', C=1))
])

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Tehnopark/porn_data/train_data.csv')
df_train, df_test = train_test_split(df, train_size = 0.8, random_state=42)
model.fit(df_train, df_train.target)
pred_train = model.predict(df_train)
pred_test = model.predict(df_test)
print("pred_train: ", f1_score(df_train.target.values, pred_train))
print("pred_test: ", f1_score(df_test.target.values, pred_test))

pred_train:  0.9891970152578238
pred_test:  0.9733194216723804


Предсказываю моделью обученной на собранной выборке исходную выборку

In [ ]:
no_porn_df = pd.read_csv('/content/drive/MyDrive/Tehnopark/porn_data/my_no_porn_data.csv')
porn_df = pd.read_csv('/content/drive/MyDrive/Tehnopark/porn_data/my_porn_data.csv')

no_porn_df = no_porn_df[{'url_tokens', 'title_tokens', 'target'}]
porn_df = porn_df[{'url_tokens', 'title_tokens', 'target'}]
df = df[{'url_tokens', 'title_tokens', 'target'}]
df_train = pd.concat([porn_df, no_porn_df])

model.fit(df_train, df_train.target)
pred_train = model.predict(df_train)
pred_test = model.predict(df)
print("pred_train: ", f1_score(df_train.target.values, pred_train))
print("pred_test: ", f1_score(df.target.values, pred_test))

pred_train:  0.9983939480359635
pred_test:  0.38038847404828435


Вывод: не сложилось. Результаты оч низкие, даже бессмыслено проверять признаки по отдельности

Проверяю, стоит ли добавлять выборку только порно наблюдениями. Так как на мой взгляд данные очень похожи. Для правдивости эксперимента разыгрываю ситуацию,при которой тестовая выборка соизмерима с тренировочной. 

In [ ]:
df = df[{'url_tokens', 'title_tokens', 'target'}]
df.head()

,target,url_tokens,title_tokens
0,0,m kp md,экс министр экономика молдова глава мидэи цель...
1,0,kp by,песня стать известный многий телезритель благо...
2,0,fan se rials tv,банша сезон серия бремя красота смотреть онлайн
3,0,co lor box spb,бесить картинка
4,0,tu la sport,новомосковск сыграть следж хоккеист алексински...


In [ ]:
from sklearn.model_selection import train_test_split
from ast import literal_eval
df_train, df_test = train_test_split(df, random_state=42, train_size=0.40, shuffle = True)

In [ ]:
preprocessor = ColumnTransformer([
    ('url_tokens', TfidfVectorizer(analyzer='word'), 'url_tokens'),
    ('title_tokens', TfidfVectorizer(analyzer='word'), 'title_tokens'),
])

model = Pipeline([
    ('preprocessor', preprocessor),
    ('Linear_model', LogisticRegression(class_weight='balanced', C=1))
])


model.fit(df_train, df_train.target)

from sklearn.metrics import f1_score
pred_train = model.predict(df_train)
pred_test = model.predict(df_test)
print('train f1_score: ', f1_score(pred_train, df_train.target))
print('test f1_score: ', f1_score(pred_test, df_test.target))

train f1_score:  0.9901169370049038
test f1_score:  0.9633502688304642


Теперь дополню своими данными

In [ ]:
df_train = pd.concat([df_train, porn_df], ignore_index = True)
df_train

,target,url_tokens,title_tokens
0,0,go lov bukh ua,позичка автомоб зособь область оподаткуванний
1,0,mo loc hni sa,тержинан молочница лечение отзыв инструкция
2,0,blast beat shop,болт барабан dixon pats
3,0,m ba by blog,казаться грибок запись пользователь мама marin...
4,0,dom mo da ua,красивый женский блузка кружево купить грн маг...
...,...,...,...
154118,1,arc hi balds,сиська связать накачаны
154119,1,free hard co re porn vi deos pics aa rans free...,горячий блондинка
154120,1,se xo le,бывший давать голова день
154121,1,pus sys tri ke,волна унижение джанет


In [ ]:
preprocessor = ColumnTransformer([
    ('url_tokens', TfidfVectorizer(analyzer='word'), 'url_tokens'),
    ('title_tokens', TfidfVectorizer(analyzer='word'), 'title_tokens'),
])

model = Pipeline([
    ('preprocessor', preprocessor),
    ('Linear_model', LogisticRegression(class_weight='balanced', C=1))
])


model.fit(df_train, df_train.target)

from sklearn.metrics import f1_score
pred_train = model.predict(df_train)
pred_test = model.predict(df_test)
print('train f1_score: ', f1_score(pred_train, df_train.target))
print('test f1_score: ', f1_score(pred_test, df_test.target))

train f1_score:  0.9917809772134679
test f1_score:  0.9294762898607


Вывод: прироста не дало, к сожалению. Возможно, причина в похожести данных. Например, брались из одного сайта или собираюсь одним алгоритмом, о чем можно судить по коррелированному шуму.

# Модель

## Ансамбль линейных моделей

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import VotingClassifier


preprocessor1 = ColumnTransformer([
    ('url_tokens', TfidfVectorizer(analyzer='word', dtype = np.float32 ), 'url_tokens'),
    ('title_tokens', TfidfVectorizer(analyzer='word', dtype = np.float32), 'title_tokens'),   
])
preprocessor2 = ColumnTransformer([
    ('url_tokens', TfidfVectorizer(analyzer='word', dtype = np.float32), 'url_tokens'),
    ('title_tokens', TfidfVectorizer(analyzer='word', dtype = np.float32), 'title_tokens'),   
])
preprocessor3 = ColumnTransformer([
    ('url', CountVectorizer(analyzer='char_wb', ngram_range = (2, 4), dtype = np.float32, max_features=100000), 'url'),
    ('title', CountVectorizer(analyzer='char_wb',ngram_range = (2, 4), dtype = np.float32, max_features=100000), 'title')  
])
preprocessor4 = ColumnTransformer([
    ('url', CountVectorizer(analyzer = 'char_wb', ngram_range = (2, 4), dtype = np.float32, max_features=100000), 'url'),   
    ('title', CountVectorizer(analyzer = 'char_wb', ngram_range = (2, 4), dtype = np.float32, max_features=100000), 'title')
])

    


model1 = Pipeline([
    ('preprocessor', preprocessor1),
    ('linear_model', LogisticRegressionCV(class_weight = 'balanced', cv = 3))
])
model2 = Pipeline([
    ('preprocessor', preprocessor2),
    ('linear_model', LogisticRegressionCV(class_weight = 'balanced', cv = 3))
])
model3 = Pipeline([
    ('preprocessor', preprocessor3),
    ('linear_model', LogisticRegressionCV(class_weight = 'balanced', cv = 3))
])
model4 = Pipeline([
    ('preprocessor', preprocessor4),
    ('linear_model', LogisticRegressionCV(class_weight = 'balanced', cv = 3))
])


ensamble = VotingClassifier(estimators=[('mod1', model1), ('mod2', model2),
                                     ('mod3', model3), ('mod4', model4)], voting = 'soft')

Ячейка для теста отдельных моделей

In [ ]:
# model = model3
# df = pd.read_csv('train_data.csv')
# df_train, df_test = train_test_split(df, train_size=0.8, random_state=42, shuffle=True)
# y_train = df_train.target.values.astype(np.float32)
# y_test = df_test.target.values.astype(np.float32)
# model.fit(df_train, y_train)
# pred_train = model.predict(df_train)
# pred_test = model.predict(df_test)
# print('train f1_score: ', f1_score(pred_train, y_train))
# print('test f1_score: ', f1_score(pred_test, y_test))

Обучаю ансамбль

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Tehnopark/porn_data/train_data.csv')
df_train, df_test = train_test_split(df, train_size=0.8, random_state=42, shuffle=True)
y_train = df_train.target.values.astype(np.float32)
y_test = df_test.target.values.astype(np.float32)
ensamble.fit(df_train, y_train)
pred_train= ensamble.predict(df_train)
pred_test = ensamble.predict(df_test)
print('train f1_score: ', f1_score(pred_train, y_train))
print('test f1_score: ', f1_score(pred_test, y_test))

train f1_score:  0.9995889540749598
test f1_score:  0.9879735417919423


## Word2vec + catboost

In [ ]:
from gensim.models import Word2Vec
from catboost import CatBoostClassifier

def get_dict(df):
    df = df.reset_index()
    url = []
    title = []
    for i in tqdm(range(df.shape[0])):
        url.append(df.url_tokens[i].split(' '))
        title.append(df.title_tokens[i].split(' '))
    return {'url': url, 'title': title}

def embed_text(text, word2index, word_embeddings):
    res = np.array([word_embeddings[word2index[word]] for word in text if word in word2index]).mean(0, keepdims=True)
    if res.ndim == 1:
        res = word_embeddings.mean(0, keepdims=True)
    return res

In [ ]:
test = get_dict(df_test) 
train = get_dict(df_train) 

  0%|          | 0/27062 [00:00<?, ?it/s]

  0%|          | 0/108247 [00:00<?, ?it/s]

In [ ]:
word2vec_url = Word2Vec(size=50, sg=0, window=5, min_count=2, negative=10)
word2vec_url.build_vocab(train['url'])
word2vec_url.train(train['url'], total_examples=word2vec_url.corpus_count, epochs=30)
embeddings_url = word2vec_url.wv.vectors
word2index_url = {word: i for i, word in enumerate(word2vec_url.wv.index2word)}
train_embedings_url = np.concatenate([embed_text(doc, word2index_url, embeddings_url) for doc in train['url']])
test_embedings_url = np.concatenate([embed_text(doc, word2index_url, embeddings_url) for doc in test['url']])

In [ ]:
word2vec_url.wv.most_similar('porn')

[('noxxx', 0.43063783645629883),
 ('hle', 0.38666531443595886),
 ('porns', 0.37019485235214233),
 ('jav', 0.3686683475971222),
 ('viesxxx', 0.3616902530193329),
 ('s24', 0.35834482312202454),
 ('dels', 0.3566586375236511),
 ('uh', 0.3531017303466797),
 ('snes', 0.3511388897895813),
 ('dy', 0.3484186828136444)]

In [ ]:
word2vec_title = Word2Vec(size=200, sg=0, window=5, min_count=2, negative=10)
word2vec_title.build_vocab(train['title'])
word2vec_title.train(train['title'], total_examples=word2vec_title.corpus_count, epochs=30)
embeddings_title = word2vec_title.wv.vectors
word2index_title = {word: i for i, word in enumerate(word2vec_title.wv.index2word)}
train_embedings_title = np.concatenate([embed_text(doc, word2index_title, embeddings_title) for doc in train['title']])
test_embedings_title = np.concatenate([embed_text(doc, word2index_title, embeddings_title) for doc in test['title']])

In [ ]:
word2vec_title.wv.most_similar('фильм')

[('мультфильм', 0.5510241389274597),
 ('драма', 0.5314066410064697),
 ('кино', 0.5033471584320068),
 ('дорамы', 0.4639444053173065),
 ('боевик', 0.4614221751689911),
 ('комедия', 0.4448634684085846),
 ('актриса', 0.44398877024650574),
 ('вильгельм', 0.44392889738082886),
 ('сериал', 0.44200602173805237),
 ('джессика', 0.4407733976840973)]

In [ ]:
x_train = np.hstack([train_embedings_title, train_embedings_url])
x_test = np.hstack([test_embedings_title, test_embedings_url])

In [ ]:
boost_model = CatBoostClassifier(random_seed=42)
boost_model.fit(x_train, y_train, verbose = False)
pred_train = boost_model.predict(x_train)
pred_test = boost_model.predict(x_test)
print('train f1_score: ', f1_score(pred_train, y_train))
print('test f1_score: ', f1_score(pred_test, y_test))

train f1_score:  0.9962501874906254
test f1_score:  0.9725158562367865


In [ ]:
p_train_boost = boost_model.predict_proba(x_train)
p_test_boost = boost_model.predict_proba(x_test)
ensamble_train = ensamble.predict_proba(df_train)
ensamble_test = ensamble.predict_proba(df_test)
res_train = p_train_boost * 0.3 + 0.7 * ensamble_train
res_test = p_test_boost * 0.3 + 0.7 *ensamble_test
finaly_train = np.argmax(res_train, axis = 1)
finaly_test = np.argmax(res_test, axis = 1)
print('train f1_score: ', f1_score(finaly_train, y_train))
print('test f1_score: ', f1_score(finaly_test, y_test))

train f1_score:  0.9995515359892369
test f1_score:  0.9862931164331977


Обучаю на всех данных

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Tehnopark/porn_data/train_data.csv')
df_ = pd.read_csv('/content/drive/MyDrive/Tehnopark/porn_data/test_data.csv')
y_train = df.target.values.astype(np.float32)
order = {'url', 'url_tokens', 'title', 'title_tokens'}
df = df[order]
df_ = df_[order]

In [ ]:
ensamble.fit(df, y_train)
pred_train = ensamble.predict(df)
pred_test = ensamble.predict(df_)
print('train f1_score: ', f1_score(pred_train, y_train))

train f1_score:  0.9995813897859108


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Tehnopark/porn_data/test_data.csv')
submit = submit.iloc[:, :1]
submit['target'] = pred_test.astype(bool)
submit.to_csv('/content/drive/MyDrive/Tehnopark/porn_data/submit.csv', index = False)

In [ ]:
test = get_dict(df_) 
train = get_dict(df)

word2vec_url = Word2Vec(size=100, sg=0, window=5, min_count=2, negative=10)
word2vec_url.build_vocab(train['url'])
word2vec_url.train(train['url'], total_examples=word2vec_url.corpus_count, epochs=30)
embeddings_url = word2vec_url.wv.vectors
word2index_url = {word: i for i, word in enumerate(word2vec_url.wv.index2word)}
train_embedings_url = np.concatenate([embed_text(doc, word2index_url, embeddings_url) for doc in train['url']])
test_embedings_url = np.concatenate([embed_text(doc, word2index_url, embeddings_url) for doc in test['url']])


word2vec_title = Word2Vec(size=200, sg=0, window=5, min_count=2, negative=10)
word2vec_title.build_vocab(train['title'])
word2vec_title.train(train['title'], total_examples=word2vec_title.corpus_count, epochs=30)
embeddings_title = word2vec_title.wv.vectors
word2index_title = {word: i for i, word in enumerate(word2vec_title.wv.index2word)}
train_embedings_title = np.concatenate([embed_text(doc, word2index_title, embeddings_title) for doc in train['title']])
test_embedings_title = np.concatenate([embed_text(doc, word2index_title, embeddings_title) for doc in test['title']])

x_train = np.hstack([train_embedings_title, train_embedings_url])
x_test = np.hstack([test_embedings_title, test_embedings_url])

In [ ]:
boost_model = CatBoostClassifier(random_seed=42)
boost_model.fit(x_train, y_train, verbose = False)
boost_pred_train = boost_model.predict(x_train)
print('train f1_score: ', f1_score(boost_pred_train, y_train))

train f1_score:  0.996158693955945


In [ ]:
ensamble_train_p = ensamble.predict_proba(df)
ensamble_test_p = ensamble.predict_proba(df_)

p_train_boost = boost_model.predict_proba(x_train)
p_test_boost = boost_model.predict_proba(x_test)


res_train = p_train_boost * 0.3 + 0.7 * ensamble_train_p
res_test = p_test_boost * 0.3 + 0.7 * ensamble_test_p

finaly_train = np.argmax(res_train, axis = 1)
finaly_test = np.argmax(res_test, axis = 1)
print('train f1_score: ', f1_score(finaly_train, y_train))

train f1_score:  0.9995514219922844


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Tehnopark/porn_data/test_data.csv')
submit = submit.iloc[:, :1]
submit['target'] = finaly_test.astype(bool)
submit.to_csv('submit2.csv', index = False)